In [ ]:
import os
import re
import sys
import numpy as np

from scipy import interpolate
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D as ax3

In [ ]:
sys.path.insert(0, '..')
from dragonfly_automation import utils

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# The list of positions generated by the HCS Site Generator plugin
position_list_filepath = \
    '/Users/keith.cheveralls/image-data/dragonfly-automation-tests/ML0000-20191009-2/HCS_sites_20191009.pos'

### Old interpolation method

Here, we just measure the positions of the FocusDrive at the corners of the ROI.

In [ ]:
# parameters required for z-position interpolation
num_rows = 6
num_columns = 8
num_positions_per_well = 25

corner_z_positions = (
    (7500,  7400),  # B2, B9
    (7450,  7350),  # G2, G9
)

In [ ]:
# interpolate the positions
new_position_list_filepath, new_position_list = utils.interpolate_stage_positions_from_corners(
    position_list_filepath,
    (num_rows, num_columns),
    num_positions_per_well,
    corner_z_positions)

new_position_list_filepath

### New interpolation method

Here, we measure the focusdrive position at any number of wells and use all of them to interpolate. 

In [ ]:
# define the ROI by specifying the top left and bottom right wells 
top_left_well_id = 'B2'
bottom_right_well_id = 'G9'

# arbitrary list of measured positions
measured_focusdrive_positions = {
    'B9': 7600,
    'B5': 7650,
    'B2': 7500,
    'E2': 7450,
    'G2': 7400,
    'G5': 7500,
    'G9': 7450,
    'E9': 7550,
    'E5': 7500,
}

### Preview the interpolation

In [ ]:
measured_positions = np.array([
    (*utils.well_id_to_position(well_id), zpos) 
        for well_id, zpos in measured_focusdrive_positions.items()])

In [ ]:
interpolator = interpolate.interp2d(
    measured_positions[:, 0], 
    measured_positions[:, 1], 
    measured_positions[:, 2], 
    kind='linear')

In [ ]:
topl_x, topl_y = utils.well_id_to_position(top_left_well_id)
botr_x, botr_y = utils.well_id_to_position(bottom_right_well_id)

In [ ]:
x = np.linspace(topl_x, botr_x, 50)
y = np.linspace(topl_y, botr_y, 50)
X, Y = np.meshgrid(x, y)
Z = interpolator(x, y)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(
    X, Y, Z, rstride=1, cstride=1,
    cmap='viridis', edgecolor='none')

ax.scatter3D(
    measured_positions[:, 0], measured_positions[:, 1], measured_positions[:, 2], color='red')

### Interpolate the position list

In [ ]:
new_position_list_filepath, new_position_list = utils.interpolate_stage_positions_from_all(
    position_list_filepath,
    measured_focusdrive_positions,
    top_left_well_id,
    bottom_right_well_id)

### Visualize the results

In [ ]:
def xyz_from_pos(pos):

    well_id, site_num = utils.parse_hcs_site_label(pos['LABEL'])
    x, y = utils.well_id_to_position(well_id)
    
    focusdrive = [d for d in pos['DEVICES'] if d['DEVICE']=='FocusDrive'][0]
    z = focusdrive['X']
    
    return x, y, z

In [ ]:
pos = np.array([xyz_from_pos(p) for p in new_position_list['POSITIONS']])
pos.shape

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.scatter3D(pos[:, 0], pos[:, 1], pos[:, 2], color='gray')
ax.scatter3D(measured_positions[:, 0], measured_positions[:, 1], measured_positions[:, 2], color='red')

In [ ]:
fig = plt.figure()
ax = plt.subplot()

ax.scatter(
    pos[:, 0], 
    pos[:, 1], 
    np.abs(pos[:, 2] - 7500) + 10, 
    color='gray')

ax.scatter(
    measured_positions[:, 0], 
    measured_positions[:, 1], 
    np.abs(measured_positions[:, 2] - 7500) + 10, 
    color='red')